In [1]:
import camelot
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter    
from openpyxl import Workbook

C:\Users\SaiDheerajPeketi\anaconda3\envs\venv\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
pdf_file_name = 'test.pdf'
excel_file_name = 'extracted_tables.xlsx'
bank_name = 'Bank0'

In [3]:
tables = camelot.read_pdf(pdf_file_name, pages='all', flavor='stream')

Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)


In [4]:
print(tables)

<TableList n=6>


In [33]:
# Create a new csv file and write tables to separate csv files
for index, table in enumerate(tables):
    table.df.to_csv('table_' + str(index) + '.csv', index=False)

In [34]:
# Create a new Excel workbook and write tables to separate sheets
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
    for index, table in enumerate(tables):
        # Convert table to DataFrame
        df = table.df
        
        # Write each DataFrame to a different sheet
        sheet_name = f'Table_{index + 1}'
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Access the worksheet through writer.sheets
        worksheet = writer.sheets[sheet_name]
        
        # Adjust column widths
        for col_idx, col_cells in enumerate(df.columns, start=1):
            max_length = max(df[col_cells].astype(str).apply(len).max(), col_cells)
            adjusted_width = max_length + 2  # Adding some padding
            worksheet.column_dimensions[get_column_letter(col_idx)].width = adjusted_width

print(f"All tables have been extracted and saved to {excel_file_name}.")

All tables have been extracted and saved to extracted_tables.xlsx.


## Bank 0

In [5]:
user_name = 'SAI DHEERAJ PEKETI'

In [6]:
import re

def is_valid_date(text):
    date_pattern = r'^\d{2}-\d{2}-\d{4}$'
    return bool(re.match(date_pattern, text))

def is_valid_date_or_empty(text):
    date_pattern = r'^\d{2}-\d{2}-\d{4}$'
    return bool(re.match(date_pattern, text)) or text == ''

In [14]:
new_rows = []
for index, table in enumerate(tables):
    if table.df.iloc[0, 0] == user_name:
        print(f'Table_{index + 1}: {table.df.shape[1]}')
        current_table_df = table.df.iloc[2:].reset_index(drop=True)
        print(current_table_df.columns)
        table_columns = ['DATE', 'NARRATION', 'CHQ.NO.', 'WITHDRAWAL (DR)', 'DEPOSIT (CR)', 'BALANCE']
        if table.df.shape[1] == 7:
            current_table_df.loc[current_table_df[1] != '', 2] = current_table_df[1]
            current_table_df.drop([1], axis=1, inplace=True)
        elif table.df.shape[1] == 6:
            invalid_dates = ~current_table_df[0].apply(is_valid_date_or_empty)
            current_table_df.loc[invalid_dates, 1] = current_table_df.loc[invalid_dates, 0]
            current_table_df.loc[invalid_dates, 0] = ''
        current_table_df.reset_index(drop=True, inplace=True)
        current_table_df.columns = table_columns
        isFirst = False
        i = 0
        
        while i < len(current_table_df):
            if is_valid_date(current_table_df.iloc[i, 0]):
                isFirst = True
                new_rows.append({
                    'DATE': current_table_df.iloc[i, 0],
                    'NARRATION': current_table_df.iloc[i, 1],
                    'CHQ.NO.': current_table_df.iloc[i, 2],
                    'WITHDRAWAL (DR)': current_table_df.iloc[i, 3],
                    'DEPOSIT (CR)': current_table_df.iloc[i, 4],
                    'BALANCE': current_table_df.iloc[i, 5]
                })
                i += 1
            else:
                if i + 1 < len(current_table_df) and i + 2 < len(current_table_df):
                    new_rows.append({
                        'DATE': current_table_df.iloc[i + 1, 0],
                        'NARRATION': current_table_df.iloc[i, 1] + " " + current_table_df.iloc[i + 2, 1],
                        'CHQ.NO.': current_table_df.iloc[i + 1, 2],
                        'WITHDRAWAL (DR)': current_table_df.iloc[i + 1, 3],
                        'DEPOSIT (CR)': current_table_df.iloc[i + 1, 4],
                        'BALANCE': current_table_df.iloc[i + 1, 5]
                    })
                i += 3
if len(new_rows) != 0:
    table_df = pd.DataFrame(new_rows)
    table_df['WITHDRAWAL (DR)'] = pd.to_numeric(table_df['WITHDRAWAL (DR)'], errors='coerce')
    table_df['DEPOSIT (CR)'] = pd.to_numeric(table_df['DEPOSIT (CR)'], errors='coerce')
    
    # Write to Excel file
    file_name = 'table.xlsx'
    table_df.to_excel(file_name, index=False, engine='openpyxl')
    workbook = load_workbook(file_name)
    worksheet = workbook.active
    
    for col_idx, col_name in enumerate(table_df.columns, start=1):
        max_length = max(
            table_df[col_name].astype(str).apply(len).max(),
            len(col_name)
        )
        adjusted_width = max_length + 2  # Add padding
        worksheet.column_dimensions[get_column_letter(col_idx)].width = adjusted_width
    
    workbook.save(file_name)

Table_2: 7
RangeIndex(start=0, stop=7, step=1)
Table_3: 6
RangeIndex(start=0, stop=6, step=1)
Table_4: 7
RangeIndex(start=0, stop=7, step=1)
